#  Submit Molecule Geometry Optimization

In [ ]:
%aiida

In [ ]:
# General imports
import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from IPython.display import display, clear_output, HTML

# AiiDA imports
from aiida.orm import SinglefileData
from aiidalab_widgets_base import CodeDropdown
from aiida_cp2k.workchains.base import Cp2kBaseWorkChain

# Custom imports
from apps.surfaces.structure_browser import StructureBrowser
from apps.surfaces.widgets import find_mol
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.computer_code_selection import ComputerCodeDropdown
from apps.surfaces.widgets.dft_details_dev import DFTDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.cp2k_input_validity import input_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
from apps.surfaces.widgets.submit_button_dev import SubmitButton
from apps.surfaces.widgets.metadata import MetadataWidget
from apps.surfaces.widgets.get_cp2k_input_dev import Get_CP2K_Input

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
## GENERAL PARAMETERS
job_details = {'workchain':'MoleculeOptWorkChain'}
job_details['calc_type']='Full DFT'
slab_analyzed = None
atoms = None

In [ ]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    global job_details, slab_analyzed, atoms
    if not struct_browser.results.value:
        return
    atoms = struct_browser.results.value.get_ase()
    atoms.pbc = [1, 1, 1]
    slab_analyzed = find_mol.analyze_slab(atoms)
    job_details['slab_analyzed'] = slab_analyzed
    job_details['elements']      = slab_analyzed['all_elements']
    cell=job_details['slab_analyzed']['the_cell']
    cell=str(cell[0][0])+' '+str(cell[1][1])+' '+str(cell[2][2])    
    dft_details_widget.reset(cell=cell)
    
    #guess_calc_params(slab_analyzed)
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")

In [ ]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##STRUCTURE
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value') 

##VIEWER
viewer_widget = ViewerDetails()

mol_ids_info_out = ipw.Output()

display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

##CODE
#computer_code_dropdown = ComputerCodeDropdown(job_details=job_details,input_plugin='cp2k')
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')
##DFT
dft_details_widget = DFTDetails(job_details           = job_details,
                                widgets_disabled      = {
                                    'calc_type'     : True,
                                    'center_switch' : False,
                                    'charge'        : False,
                                    'multiplicity'  : False,
                                    'uks_switch'    : False,
                                    'cell'          : False
                                }
                               )    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
#dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

metadata_widget = MetadataWidget()

def widgets_values():
    global job_details, slab_analyzed, atoms
    
    # use builder to construct the input parameters
    input_builder = Cp2kBaseWorkChain.get_builder()

    
    # code
    input_builder.cp2k.code = computer_code_dropdown.selected_code
    # structure
    input_builder.cp2k.file.input_xyz = make_geom_file(struct_browser.results.value, "mol.xyz")
    
    
    
    # TODO: Carlo, fix this 
    # from here
    dft_details_widget.read_widgets_and_update_job_details()  

    if job_details['calc_type'] != 'Full DFT':
        
        # Au potential
        input_builder.cp2k.file.pot_f = SinglefileData(file='/project/apps/surfaces/slab/Au.pot')
        mol_indexes = list(itertools.chain(*slab_analyzed['all_molecules']))
        job_details['first_slab_atom']=len(mol_indexes)+1
        job_details['last_slab_atom']=slab_analyzed['numatoms']
        input_builder.cp2k.file.mol_xyz = make_geom_file(struct_browser.results.value, "mol.xyz",selection=mol_indexes)    
    
    job_details['elements'] = slab_analyzed['all_elements']

    can_submit=input_is_valid(job_details=job_details)
    
    cell_ase = atoms.cell.flatten().tolist()
    if 'cell' in job_details:
        if job_details['cell'] == '' or job_details['cell'] == None :
            job_details['cell'] = cell_ase   
        else:
            cell_abc=job_details['cell'].split()
            job_details['cell']=np.diag(np.array(cell_abc, dtype=float)).flatten().tolist()
    else:
        job_details['cell'] = cell_ase

  
    input_builder.cp2k.metadata = metadata_widget.dict
    num_machines=metadata_widget.dict['options']['resources']['num_machines']
    num_mpiprocs_per_machine=metadata_widget.dict['options']['resources']['num_mpiprocs_per_machine']
    job_details['mpi_tasks']=num_machines*num_mpiprocs_per_machine
    
  
    
    # Finalize cp2k input parameters
    input_builder.cp2k.parameters = Dict(dict=Get_CP2K_Input(input_dict = job_details).inp)

    return input_builder


#display code dropdown
display(ipw.VBox([computer_code_dropdown, dft_details_widget, metadata_widget]))

#display submit button
btn_submit = SubmitButton(workchain=Cp2kBaseWorkChain,widgets_values=widgets_values)
display(btn_submit)